In [1]:
# Global Imports
import pandas as pd

In [2]:
CSV_CUST_INFO = "TIB_CaseStudyCust.csv"
CSV_CUST_TRAN = "TIB_CaseStudyTransacts.csv"

In [3]:
def data_scrub(df, column_name: str, min_range: int = None, max_range: int = None, include_nan: bool = True):    
    # Create a filter for the DataFrame
    filter_condition = True  # Start with True

    if min_range is not None:
        filter_condition &= (df[column_name] >= min_range)
    
    if max_range is not None:
        filter_condition &= (df[column_name] <= max_range)

    if include_nan:
        filter_condition |= df[column_name].isna()
    else:
        filter_condition &= ~df[column_name].isna()  # Exclude NaN if not included

    # Apply the filter to the global DataFrame
    return df[filter_condition]


In [4]:
def cust_info_clean():
    df = pd.read_csv(CSV_CUST_INFO)
    df = df.drop('INSTITUTION_ID', axis=1)
    df = df.drop_duplicates(subset='CUSTOMER_ID')
    
    # The legal adult age is 18. The avg life expectancy is 80
    # Scrap unrealistic age values
    df = data_scrub(df, "AGE", 18, 80)

    # Store the list of valid US state codes along with D.C.
    state_codes = [
        'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
        'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
        'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
        'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
        'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY',
        'DC', '' # Include D.C.
    ]
    df = df[df["STATE"].isin(state_codes)]

    # We assume a minimum of 1 TOTAL ACCOUNT with the institution
    df = data_scrub(df, "TOTAL_ACCOUNTS", 1, 50)
    
    # Estimated Y2D income amount threshold
    df = data_scrub(df, "YEAR_TO_DATE_ESTIMATED_INCOME_AMOUNT", 1, None)
    
    df.to_csv('Cust_Filter.csv', index=False)

In [19]:
def cust_tran_clean():
    df = pd.read_csv(CSV_CUST_TRAN)
    df = df.drop('INSTITUTION_ID', axis=1)
    df = df.drop_duplicates(subset='TRANSACTION_ID')
    # df = df.dropna(subset=['TRANSACTION_AMOUNT'])
    df = data_scrub(df, "TRANSACTION_AMOUNT", include_nan=False)
    df.to_csv('Tran_Filter.csv', index=False)

In [ ]:
cust_info_clean()

In [20]:
cust_tran_clean()